In [1]:
"""This Notebook will seek to compare the multiple ways available of calculating diffusion coefficients.  Elizabeth's
method looks at the local derivative of the diffusion coefficient.  While interesting in some respects, it can 
propagate error very quickly.  Additional methods I want to look at include: 

Unmodified linear regression (doesn't take into account error bars of individual points through covariance matrix)
Weighted linear regression (weighs each point by its accuracy, and maybe takes into account errorbars in cov?)
"""

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify

import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
# Type in datasets you would like to combine.  Adjust totvids to reflect the total number of videos. Also type in the
# umppx and fps in the conversion dictionary.

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.30, 4.84, 1)
conversion[2] = (0.30, 4.84, 1)
conversion[3] = (0.30, 5.02, 1)

trajectory = dict()

trajectory[1] = np.genfromtxt('../../sample_data/12-07-06-75p-glycerol-exp/CON_40x_2D_200ms_2.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('../../sample_data/12-07-06-75p-glycerol-exp/CON_40x_2D_200ms_3_T.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('../../sample_data/12-07-06-75p-glycerol-exp/CON_40x_2D_200ms_4_T.csv',
            delimiter =",")

totvids = 3

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

#Get rid of trajectories that are too short, as determined by prettify
final = dict()
tots = dict()

for num in range(1, totvids + 1):
    (final[num], tots[num]) = prettify(trajectory[num], 40, 40, conversion[num][0], conversion[num][1], conversion[num][2])


#Collect trajectories of separate videos into one array
parts = dict()
lehi = dict()

for num in range(1, totvids + 1):
    parts[num] = tots[num]
    lehi[num] = final[num][1]
    counter = 1

    while counter < parts[num]:
        counter = counter + 1
        lehi[num] = np.append(lehi[num], final[num][counter], axis=0)

for num in range(2, totvids + 1):
    lehi[num][:,0] = lehi[num][:,0] + max(lehi[num-1][:, 0])        
        
nephi = lehi[1]
counter = 1

while counter < totvids:
    counter = counter + 1
    nephi = np.append(nephi, lehi[counter], axis=0)

In [3]:
def Dpointder(traj, n1, n2, n3, tn, nD):
    """
    Outputs the average diffusion coefficient at a desired timepoint.  User puts in a time.  The code finds the point
    closest to that time and gives the diffusion coefficient at that time.
    
    Note that this code is only for 2D datasets.  I need to modify it slightly for 3D datasets.

    n1: particle numbers (typically 0)
    n2: time (15 when using prettify)
    n3: MSDs or Deffs (9 for MSDs 17 for Deffs)
    tn: desired time
    nD: Either '2D', '1Dx', or '1Dy'
    
    Returns the values:
    
    MMSD: The diffusion coefficient at the desired time
    SD: The standard deviation
    total1: The number of particles taken into account in the calculation
    t: The timepoint at which the diffusion coefficient was calculated
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1
    SD = np.zeros(np.shape(MMSD))
    t = time[1][:]
    
    #Now to calculate the standard dev at each point:
    for num in range (1, total1):
        SDunit = (MSD[num] - MMSD)**2
        SD = SD + SDunit
    SD = np.sqrt(SD/total1)
    SE = SD/np.sqrt(total1)
    
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx
    
    td, idx = find_nearest(t, tn)
    
    if nD == '2D':  
        D = MMSD[idx, 0]
        SDd = SD[idx, 0]
    elif nD == '1Dx':
        D = MMSD[idx, 1]
        SDd = SD[idx, 1]
    else:
        D = MMSD[idx, 2]
        SDd = SD[idx, 2]
    
    return D, SDd, total1, td

In [8]:
D = dict()
SD = dict()
total = dict()
t = dict()

D['pd2D'], SD['pd2D'], total['pd2D'], t['pd2D'] = Dpointder(nephi, 0, 15, 17, 1, '2D')
D['pd1Dx'], SD['pd1Dx'], total['pd1Dx'], t['pd1Dx'] = Dpointder(nephi, 0, 15, 17, 1, '1Dx')
D['pd1Dy'], SD['pd1Dy'], total['pd1Dy'], t['pd1Dy'] = Dpointder(nephi, 0, 15, 17, 1, '1Dy')

In [9]:
print(D['pd2D'])
print(D['pd1Dx'])
print(D['pd1Dy'])

0.124185570706
0.104401956486
0.143969184927


In [10]:
total['pd2D']

122

In [11]:
def Dlinalg(traj, n1, n2, n3, nD):
    """
    Outputs the average diffusion coefficient at a desired timepoint.  User puts in a time.  The code finds the point
    closest to that time and gives the diffusion coefficient at that time.
    
    Note that this code is only for 2D datasets.  I need to modify it slightly for 3D datasets.

    n1: particle numbers (typically 0)
    n2: time (15 when using prettify)
    n3: MSDs or Deffs (9 for MSDs 17 for Deffs) **MUST be in MSDs in this case, not Deffs**
    nD: Either '2D', '1Dx', or '1Dy'
    
    Returns the values:
    
    MMSD: The diffusion coefficient at the desired time
    SD: The standard deviation
    total1: The number of particles taken into account in the calculation
    t: The timepoint at which the diffusion coefficient was calculated
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1
    SD = np.zeros(np.shape(MMSD))
    t = time[1][:]
    
    #Now to calculate the standard dev at each point:
    for num in range (1, total1):
        SDunit = (MSD[num] - MMSD)**2
        SD = SD + SDunit
    SD = np.sqrt(SD/total1)
    SE = SD/np.sqrt(total1)

    w = dict()
    cov = dict()
    sd = dict()
    line = dict()
    #A = np.ones((np.shape(t)[0], 2))
    #A[:, 0] = t
    #w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
    #w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
    #w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
    
    w[0], cov[0] = np.polyfit(t, MMSD[:, 0], 1, cov=True)
    w[1], cov[1] = np.polyfit(t, MMSD[:, 1], 1, cov=True)
    w[2], cov[2] = np.polyfit(t, MMSD[:, 2], 1, cov=True)
    
    sd[0] = np.sqrt(np.diag(cov[0]))
    sd[1] = np.sqrt(np.diag(cov[1]))
    sd[2] = np.sqrt(np.diag(cov[2]))
    
    if nD == '2D':  
        D = w[0][0]/4
        SDd = sd[0][0]/4
    elif nD == '1Dx':
        D = w[1][0]/2
        SDd = sd[1][0]/2
    else:
        D = w[2][0]/2
        SDd = sd[2][0]/2
    
    return D, SDd, total1

In [12]:
D['linalg2D'], SD['linalg2D'], total['linalg2D'] = Dlinalg(lehi[1], 0, 15, 9, '2D')
D['linalg1Dx'], SD['linalg1Dx'], total['linalg1Dx'] = Dlinalg(lehi[1], 0, 15, 9, '1Dx')
D['linalg1Dy'], SD['linalg1Dy'], total['linalg1Dy'] = Dlinalg(lehi[1], 0, 15, 9, '1Dy')

In [13]:
print(D['linalg2D'])
print(D['linalg1Dx'])
print(D['linalg1Dy'])

0.0784968292338
0.0687181377712
0.0882755206964


In [14]:
def Dlinalgw(traj, n1, n2, n3, nD):
    """
    Outputs the average diffusion coefficient at a desired timepoint.  User puts in a time.  The code finds the point
    closest to that time and gives the diffusion coefficient at that time.
    
    Note that this code is only for 2D datasets.  I need to modify it slightly for 3D datasets.

    n1: particle numbers (typically 0)
    n2: time (15 when using prettify)
    n3: MSDs or Deffs (9 for MSDs 17 for Deffs) **MUST be in MSDs in this case, not Deffs**
    nD: Either '2D', '1Dx', or '1Dy'
    
    Returns the values:
    
    MMSD: The diffusion coefficient at the desired time
    SD: The standard deviation
    total1: The number of particles taken into account in the calculation
    t: The timepoint at which the diffusion coefficient was calculated
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1
    SD = np.zeros(np.shape(MMSD))
    t = time[1][:]
    
    #Now to calculate the standard dev at each point:
    for num in range (1, total1):
        SDunit = (MSD[num] - MMSD)**2
        SD = SD + SDunit
    SD = np.sqrt(SD/total1)
    SE = SD/np.sqrt(total1)

    w = dict()
    cov = dict()
    sd = dict()
    line = dict()
    #A = np.ones((np.shape(t)[0], 2))
    #A[:, 0] = t
    #w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
    #w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
    #w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
    
    weight = dict()
    weight[0] = [1.0 / ty for ty in SD[:, 0]]
    weight[1] = [1.0 / ty for ty in SD[:, 1]]
    weight[2] = [1.0 / ty for ty in SD[:, 2]]
    
    w[0], cov[0] = np.polyfit(t, MMSD[:, 0], 1, w=weight[0], full=False, cov=True)
    w[1], cov[1] = np.polyfit(t, MMSD[:, 1], 1, w=weight[1], full=False, cov=True)
    w[2], cov[2] = np.polyfit(t, MMSD[:, 2], 1, w=weight[2], full=False, cov=True)
    
    sd[0] = np.sqrt(np.diag(cov[0]))
    sd[1] = np.sqrt(np.diag(cov[1]))
    sd[2] = np.sqrt(np.diag(cov[2]))
    
    if nD == '2D':  
        D = w[0][0]/4
        SDd = sd[0][0]/4
    elif nD == '1Dx':
        D = w[1][0]/2
        SDd = sd[1][0]/2
    else:
        D = w[2][0]/2
        SDd = sd[2][0]/2
    
    return D, SDd, total1

In [15]:
D['linalgw2D'], SD['linalgw2D'], total['linalgw2D'] = Dlinalgw(lehi[1], 0, 15, 9, '2D')
D['linalgw1Dx'], SD['linalgw1Dx'], total['linalgw1Dx'] = Dlinalgw(lehi[1], 0, 15, 9, '1Dx')
D['linalgw1Dy'], SD['linalgw1Dy'], total['linalgw1Dy'] = Dlinalgw(lehi[1], 0, 15, 9, '1Dy')

In [16]:
print(D['linalgw2D'])
print(D['linalgw1Dx'])
print(D['linalgw1Dy'])

0.0804586371016
0.0792060002065
0.0690870226907


In [ ]:
print(SD['linalg2D'])
print(SD['linalgw2D'])